## 02 - Generate and interferogram using SNAP

### Quick link

* [Objective](#Objective)
* [Data](#Data)
* [Workflow](#Workflow)
  * [Orbit file correction](#Apply-orbit-file-SNAP-Operator)
  * [TOPSAR split](#TOPSAR-split-SNAP-Operator)
  * [Back-Geocoding](#Back-Geocoding-SNAP-Operator)
  * [Interferogram](#Interferogram-SNAP-Operator)
  * [TOPSAR Deburst](#TOPSAR-Deburst-SNAP-Operator)
  * [TOPSAR Merge](#TOPSAR-Merge-SNAP-Operator)
  * [TopoPhaseRemoval](#TopoPhaseRemoval-SNAP-Operator)
  * [GoldsteinPhaseFiltering](#GoldsteinPhaseFiltering-SNAP-Operator)
  * [Terrain correction](#Terrain-correction-SNAP-Operator)
* [Note](#Note)
* [Way forward](#Way-forward) 
* [License](#License)

### Objective 

First stage-in the coseismic interferometric pair discovered in the previous lesson.

The use the SNAP Toolbox operators with Python to create a workflow that applies the SNAP Operators:

- Orbit file correction
- TOPSAR split
- Back-Geocoding
- Interferogram
- TOPSAR Deburst
- TOPSAR Merge
- TopoPhaseRemoval
- GoldsteinPhaseFiltering
- Terrain correction

This is just an example of a processing workflow using SNAP Operators. Potentially all SNAP graphs can be performed with this approach.

### Data

SENTINEL data products are made available systematically and free of charge to all data users including the general public, scientific and commercial users. Radar data will be delivered within an hour of reception for Near Real-Time (NRT) emergency response, within three hours for NRT priority areas and within 24 hours for systematically archived data.

All data products are distributed in the SENTINEL Standard Archive Format for Europe (SAFE) format.

Data products are available in single polarisation (VV or HH) for Wave mode and dual polarisation (VV+VH or HH+HV) and single polarisation (HH or VV) for SM, IW and EW modes.

We will use the slave discovered in the previous lesson:

In [1]:
slave_product = {'enclosure': 'https://store.terradue.com/download/sentinel1/files/v1/S1A_IW_SLC__1SDV_20161018T163206_20161018T163233_013547_015AEB_712A',
 'identifier': 'S1A_IW_SLC__1SDV_20161018T163206_20161018T163233_013547_015AEB_712A',
 'productType': 'SLC',
 'self': 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20161018T163206_20161018T163233_013547_015AEB_712A',
 'startdate': '2016-10-18T16:32:06.0438950Z',
 'track': '175',
 'wkt': 'POLYGON((19.368031 40.835575,22.396507 41.235806,22.733843 39.616779,19.779461 39.21563,19.368031 40.835575))'}


In [2]:
master_product = {'enclosure': 'https://store.terradue.com/download/sentinel1/files/v1/S1A_IW_SLC__1SDV_20161006T163206_20161006T163233_013372_01554F_7FE0',
 'identifier': 'S1A_IW_SLC__1SDV_20161006T163206_20161006T163233_013372_01554F_7FE0',
 'self': 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20161006T163206_20161006T163233_013372_01554F_7FE0',
 'startdate': '2016-10-06T16:32:06.0564920Z',
 'track': '175',
 'wkt': 'POLYGON((19.367752 40.834766,22.396078 41.234989,22.733395 39.616081,19.77915 39.214939,19.367752 40.834766))'}

### Workflow

#### Import the packages required for processing the data

In [3]:
import os

from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import dateutil.parser as parser
import gc
from datetime import datetime

import cioppy

import gdal
import osr

from shapely.wkt import loads
from shapely.geometry import box

import lxml.etree as etree
import numpy as np

**Stage-in the slave and the master**

Define where the data is staged-in. 

In [4]:
data_path = '/data2'

Stage-in the products

In [5]:
for product in [slave_product, master_product]:
    
    local_products = []
    
    local_path = os.path.join(data_path, product['identifier'])
    if not (os.path.isdir(local_path)):
        retrieved = ciop.copy(product['enclosure'], data_path)
    else: 
        retrieved = local_path
        
    local_products.append(retrieved)

#### Read the products

In [6]:
s1meta = "manifest.safe"

for index, s1path in enumerate([slave_product['identifier'], master_product['identifier']]):

    s1prd= "%s/%s/%s.SAFE/%s" % (data_path, s1path, s1path, s1meta)
    reader = ProductIO.getProductReader("SENTINEL-1")
    
    if index == 0:
        slave = reader.readProductNodes(s1prd, None)
        slave_date = parser.parse(slave.getStartTime().toString()).isoformat()[:19]
    else:
        master = reader.readProductNodes(s1prd, None)
        master_date = parser.parse(slave.getStartTime().toString()).isoformat()[:19]

### Apply orbit file SNAP Operator

The orbit state vectors provided in the metadata of a SAR product are generally not accurate and can be refined with the precise orbit files which are available days-to-weeks after the generation of the product. 

The orbit file provides accurate satellite position and velocity information. Based on this information, the orbit state vectors in the abstract metadata of the product are updated.

List the parameters of the **Apply orbit file** SNAP Operator

In [7]:
operator = 'Apply-Orbit-File'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('orbitType', 'Sentinel Precise (Auto Download)')
('polyDegree', '3')
('continueOnFail', 'false')


Apply the operator:

In [8]:
parameters = HashMap()

parameters.put('orbitType', 'Sentinel Precise (Auto Download)')
parameters.put('polyDegree', '3')
parameters.put('continueOnFail', 'false')


master_orbit = GPF.createProduct(operator,
                                 parameters, 
                                 master)
    
slave_orbit = GPF.createProduct(operator,
                                 parameters, 
                                 slave)

### TOPSAR split SNAP Operator

The TOPSAR Split operator provides a convenient way to split each subswath with selected bursts into a separate product. This operator is the first processing step in the TOPS InSAR processing chain.

List the parameters of the **TOPSAR-Split** SNAP Operator

In [9]:
operator = 'TOPSAR-Split'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('subswath', None)
('selectedPolarisations', None)
('firstBurstIndex', '1')
('lastBurstIndex', '9999')
('wktAoi', None)


Apply the operator:

In [10]:
slave_split_prds = []
master_split_prds = []

for subswath in ['IW1', 'IW2', 'IW3']:  
    
    parameters = HashMap()

    parameters.put('subswath', subswath)
    parameters.put('selectedPolarisations', 'VV')
    parameters.put('firstBurstIndex', '1')
    parameters.put('lastBurstIndex', '9999')

    master_split_prds.append(GPF.createProduct(operator,
                           parameters, 
                           master_orbit))   
    
    slave_split_prds.append(GPF.createProduct(operator,
                           parameters, 
                           slave_orbit))   
    

### Back-Geocoding SNAP Operator

This operator co-registers two S-1 SLC split products (master and slave) of the same sub-swath using the orbits of the two products and a Digital Elevation Model (DEM). 

In resampling the slave images into master frame, deramp and demodulation are performed first to the slave image, then the truncated-sinc interpolation is performed. Finally, the reramp and remodulation are applied to the interpolated slave image.

List the parameters of the **Back-Geocoding** SNAP Operator

In [11]:
operator = 'Back-Geocoding'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('demName', 'SRTM 3Sec')
('demResamplingMethod', 'BICUBIC_INTERPOLATION')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('resamplingType', 'BISINC_5_POINT_INTERPOLATION')
('maskOutAreaWithoutElevation', 'true')
('outputRangeAzimuthOffset', 'false')
('outputDerampDemodPhase', 'false')
('disableReramp', 'false')


Apply the operator:

In [12]:
backgeo_prds = []

for index, subswath in enumerate(['IW1', 'IW2', 'IW3']):  
    

    parameters = HashMap()

    for param in op_params:
        parameters.put(param.getName(), param.getDefaultValue())


    backgeo_prds.append(GPF.createProduct(operator,
                           parameters, 
                           [master_split_prds[index], 
                            slave_split_prds[index]
                           ]))

### Interferogram SNAP Operator

This operator computes (complex) interferogram, with or without subtraction of the flat-earth (reference) phase. The reference phase is subtracted using a 2d-polynomial that is also estimated in this operator.

If the orbits for interferometric pair are known, the flat-earth phase is estimated using the orbital and metadata information and subtracted from the complex interferogram. The flat-earth phase is the phase present in the interferometric signal due to the curvature of the reference surface. The geometric reference system of the reference surface is defined by the reference system of satellite orbits (for now only WGS84 supported, which the reference system used by all space-borne SAR systems).

The flat-earth phase is computed in a number of points distributed over the total image, after which a 2d-polynomial is estimated (using least squares) fitting these 'observations', (e.g. plane can be fitted by setting the degree to 1.)

A polynomial of degree 5 normally is sufficient to model the reference phase for a full SAR scene (approx 100x100km). While, a lower degree might be selected for smaller images, and higher degree for 'long-swath' scenes. Note that the higher order terms of the flat-earth polynomial are usually small, because the polynomial describes a smooth, long wave body (ellipsoid). To recommended polynomial degree, that should ensure the smooth surface for most image sizes and areas of the world is 5th degree.

List the parameters of the **Interferogram** SNAP Operator

In [13]:
operator = 'Interferogram'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

parameters = HashMap()

for param in op_params:
    print(param.getName(), param.getDefaultValue())
    parameters.put(param.getName(), param.getDefaultValue())

('subtractFlatEarthPhase', 'true')
('srpPolynomialDegree', '5')
('srpNumberPoints', '501')
('orbitDegree', '3')
('includeCoherence', 'true')
('cohWinAz', '10')
('cohWinRg', '10')
('squarePixel', 'true')
('subtractTopographicPhase', 'false')
('demName', 'SRTM 3Sec')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('externalDEMApplyEGM', 'true')
('tileExtensionPercent', '100')
('outputElevation', 'false')
('outputLatLon', 'false')


Apply the operator:

In [14]:
interferogram_prds = []

for index, subswath in enumerate(['IW1', 'IW2', 'IW3']):  
    
    parameters = HashMap()

    for param in op_params:
        parameters.put(param.getName(), param.getDefaultValue())

    interferogram_prds.append(GPF.createProduct(operator,
                           parameters, 
                           backgeo_prds[index]))

#### TOPSAR Deburst SNAP Operator

For the TOPSAR IW and EW SLC products, each product consists of one image per swath per polarization. IW products have 3 swaths and EW have 5 swaths. Each sub-swath image consists of a series of bursts, where each burst was processed as a separate SLC image. The individually focused complex burst images are included, in azimuth-time order, into a single subswath image, with black-fill demarcation in between, similar to the ENVISAT ASAR Wide ScanSAR SLC products.  

For IW, a focused burst has a duration of 2.75 sec and a burst overlap of ~50-100 samples. For EW, a focused burst has a duration of 3.19 sec. Overlap increases in range within a sub- swath.  

Images for all bursts in all sub-swaths of an IW SLC product are re-sampled to a common pixel spacing grid in range and azimuth.  Burst synchronisation is ensured for both IW and EW products.  

List the parameters of the **TOPSAR Deburst** SNAP Operator

In [15]:
operator = 'TOPSAR-Deburst'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())


('selectedPolarisations', None)


Apply the operator:

In [16]:
deburst_prds = []

for index, subswath in enumerate(['IW1', 'IW2', 'IW3']):  
 
    parameters = HashMap()
    parameters.put('selectedPolarisations', 'VV')

    deburst_prds.append(GPF.createProduct(operator,
                           parameters, 
                           interferogram_prds[index]))

#### TOPSAR Merge SNAP Operator

For the TOPSAR IW and EW SLC products, each product consists of one image per swath per polarization. IW products have 3 swaths and EW have 5 swaths. Each sub-swath image consists of a series of bursts, where each burst was processed as a separate SLC image. The individually focused complex burst images are included, in azimuth-time order, into a single sub-swath image, with black-fill demarcation in between, similar to the ENVISAT ASAR Wide ScanSAR SLC products

List the parameters of the **TOPSAR Merge** SNAP Operator

In [17]:
operator = 'TOPSAR-Merge'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('selectedPolarisations', None)


Apply the operator:

In [18]:
parameters = HashMap()
parameters.put('selectedPolarisations', 'VV')

tops_merge = GPF.createProduct(operator,
                           parameters, 
                           deburst_prds)


#### TopoPhaseRemoval SNAP Operator

This operator estimates and subtracts topographic phase from the interferogram. More specifically, this operator first "radarcodes" the Digital Elevation Model (DEM) of the area of interferogram, and then subtracts it from the complex interferogram.

This operator has to be performed after the interferogram generation. It also requires an input DEM, SRTM can be used, or any other supported DEM. The DEM handling for most of elevation models, selection and download from internet of tiles covering the area of interest, interpolation, accounting for geoid undulation, etc, is performed automatically by the operator itself.

List the parameters of the **TopoPhaseRemoval** SNAP Operator

In [19]:
operator = 'TopoPhaseRemoval'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('orbitDegree', '3')
('demName', 'SRTM 3Sec')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('tileExtensionPercent', '100')
('outputTopoPhaseBand', 'false')
('outputElevationBand', 'false')
('outputLatLonBands', 'false')


Apply the operator:

In [20]:
parameters = HashMap()

for param in op_params:
    parameters.put(param.getName(), param.getDefaultValue())


topo_phase = GPF.createProduct(operator,
                  parameters, 
                  tops_merge)

#### GoldsteinPhaseFiltering SNAP Operator

Phase filtering is a preprocessing technique that greatly reduces the residues in the later on  phase unwrapping step and enhances the phase unwrapping accuracy. 

The method implemented in this operator is a nonlinear adaptive algorithm proposed by Goldstein and Werner [1] in 1998.

List the parameters of the **GoldsteinPhaseFiltering** SNAP Operator

In [21]:
operator = 'GoldsteinPhaseFiltering'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('alpha', '1.0')
('FFTSizeString', '64')
('windowSizeString', '3')
('useCoherenceMask', 'false')
('coherenceThreshold', '0.2')


Apply the operator:

In [22]:
parameters = HashMap()

for param in op_params:
    parameters.put(param.getName(), param.getDefaultValue())


goldstein = GPF.createProduct(operator,
                  parameters, 
                  topo_phase)

#### Terrain correction SNAP Operator

Due to topographical variations of a scene and the tilt of the satellite sensor, distances can be distorted in the SAR images. Image data not directly at the sensor’s Nadir location will have some distortion. 

Terrain corrections are intended to compensate for these distortions so that the geometric representation of the image will be as close as possible to the real world.

List the parameters of the **Terrain correction** SNAP Operator

In [23]:
operator = 'Terrain-Correction'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('sourceBandNames', None)
('demName', 'SRTM 3Sec')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('externalDEMApplyEGM', 'true')
('demResamplingMethod', 'BILINEAR_INTERPOLATION')
('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
('pixelSpacingInMeter', '0')
('pixelSpacingInDegree', '0')
('mapProjection', 'WGS84(DD)')
('alignToStandardGrid', 'false')
('standardGridOriginX', '0')
('standardGridOriginY', '0')
('nodataValueAtSea', 'true')
('saveDEM', 'false')
('saveLatLon', 'false')
('saveIncidenceAngleFromEllipsoid', 'false')
('saveLocalIncidenceAngle', 'false')
('saveProjectedLocalIncidenceAngle', 'false')
('saveSelectedSourceBand', 'true')
('outputComplex', 'false')
('applyRadiometricNormalization', 'false')
('saveSigmaNought', 'false')
('saveGammaNought', 'false')
('saveBetaNought', 'false')
('incidenceAngleForSigma0', 'Use projected local incidence angle from DEM')
('incidenceAngleForGamma0', 'Use projected local incidence angle from DEM')
('auxFile', 'Latest Auxiliary Fil

Apply the operator:

In [ ]:
parameters = HashMap()

for param in op_params:
    parameters.put(param.getName(), param.getDefaultValue())


terrain = GPF.createProduct(operator,
                  parameters, 
                  goldstein)

#### Write the result

In [ ]:
ProductIO.writeProduct(terrain, 
                       'interferogram.tif',
                       'GeoTIFF-BigTIFF')

### Note

This notebook computation is a long process and we suggest using the notebook interface for the development of the workflow up to the **Write the result** step.

For its execution, we recommend using a shell to run:

```bash
jupyter nbconvert --ExecutePreprocessor.timeout=86400 --to notebook --execute '02 - Generate an interferogram using SNAP.ipynb' 

```

### Way forward

The workflow above can of course be adapted to other SNAP Graphs.

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.